In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
# faiss-gpu를 쓰기 위해서는 '런타임 유형'을 'T4 GPU'로 바꿔주어야 합니다.
# %%capture --no-stderr
# !pip install python-dotenv langchain_openai langchain_experimental langchain-chroma pypdf chromadb faiss-gpu-cu12

In [ ]:
# 환경변수 설정

In [1]:
# 라이브러리 불러오기
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")
# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# <Chroma>
# 라이브러리 불러오기
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# PDF 파일 로드
# loader = PyPDFLoader("/content/drive/MyDrive/langchain-tutorial/Ch02. RAG/Data/2024_KB_부동산_보고서_최종.pdf")
loader = PyPDFLoader("./Data/2024_KB_부동산_보고서_최종.pdf")
pages = loader.load()

print("청크의 수:", len(pages))

청크의 수: 84


In [4]:
# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)

print("분할된 청크의 수:", len(splits))

분할된 청크의 수: 135


In [5]:
# 각 청크의 길이(문자 수)를 저장한 리스트 생성
chunk_lengths = [len(chunk.page_content) for chunk in splits]
max_length = max(chunk_lengths)
min_length = min(chunk_lengths)
avg_length = sum(chunk_lengths) / len(chunk_lengths)

print('청크의 최대 길이 :', max_length)
print('청크의 최소 길이 :', min_length)
print('청크의 평균 길이 :', avg_length)

청크의 최대 길이 : 1000
청크의 최소 길이 : 56
청크의 평균 길이 : 674.9481481481481


In [6]:
# 임베딩 모델 초기화
embedding_function = OpenAIEmbeddings()

# Chroma DB 생성 및 데이터 저장
# persist_directory = "/content/drive/MyDrive/langchain-tutorial/Ch02. RAG/directory"
persist_directory = "./directory"
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding_function,
    persist_directory=persist_directory
)

In [7]:
print('문서의 수:', vectordb._collection.count())

문서의 수: 135


In [8]:
# 임베딩 모델 초기화
embedding_function = OpenAIEmbeddings()

# Chroma DB 불러오기
vectordb = Chroma(
    embedding_function=embedding_function,
    persist_directory=persist_directory
)

In [ ]:
print('문서의 수:', vectordb._collection.count())

문서의 수: 135


In [10]:
# similarity_search 메서드 사용
question = "수도권 주택 매매 전망"
top_three_docs = vectordb.similarity_search(question, k=2)

for i, doc in enumerate(top_three_docs, 1):
    print(f"문서 {i}:")
    print(f"내용: {doc.page_content[:150]}...")
    print(f"메타데이터: {doc.metadata}")
    print('--' * 20)

문서 1:
내용: 8 
2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망 
 
실 등에 따른 주택 경기 불안을 이유로 매매를 망설이며 시세 대비 저렴한 매물에만 관심을 보였다. 결
국 매도자와 매수자 간 희망가격 차이로 인한 매매 거래 위축 현상은 2023년 거래 침체의 가...
메타데이터: {'producer': 'Microsoft® Word 2016', 'total_pages': 84, 'creator': 'Microsoft® Word 2016', 'moddate': '2024-03-04T15:30:01+09:00', 'title': 'Morning Meeting', 'page_label': '15', 'creationdate': '2024-03-04T15:30:01+09:00', 'source': './Data/2024_KB_부동산_보고서_최종.pdf', 'author': '손은경', 'page': 14}
----------------------------------------
문서 2:
내용: 18 
2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망 
 
그림Ⅰ-30. 수도권 입주물량과 전세가격 변동률 추이  그림Ⅰ-31. 기타지방 입주물량과 전세가격 변동률 추이 
 
 
 
자료: KB국민은행, 부동산114  자료: KB국민은행, 부동산114...
메타데이터: {'total_pages': 84, 'page': 24, 'title': 'Morning Meeting', 'producer': 'Microsoft® Word 2016', 'moddate': '2024-03-04T15:30:01+09:00', 'author': '손은경', 'creator': 'Microsoft® Word 2016', 'creationdate': '2024-03-04T15:30:01+09:00', 'page_label': '25', 'source': './Data/2024_KB_부동산_보고서_최종.pdf'}
--------------------------

In [11]:
# similarity_search_with_relevance_scores 메서드 사용
question = "수도권 주택 매매 전망"
top_three_docs = vectordb.similarity_search_with_relevance_scores(question, k=3)

for i, doc in enumerate(top_three_docs, 1):
    print(f"문서 {i}:")
    print(f"유사 점수 {doc[1]}:")
    print(f"내용: {doc[0].page_content[:150]}...")
    print(f"메타데이터: {doc[0].metadata}")
    print('--' * 20)

문서 1:
유사 점수 0.8451337337601001:
내용: 8 
2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망 
 
실 등에 따른 주택 경기 불안을 이유로 매매를 망설이며 시세 대비 저렴한 매물에만 관심을 보였다. 결
국 매도자와 매수자 간 희망가격 차이로 인한 매매 거래 위축 현상은 2023년 거래 침체의 가...
메타데이터: {'producer': 'Microsoft® Word 2016', 'page': 14, 'title': 'Morning Meeting', 'author': '손은경', 'total_pages': 84, 'creator': 'Microsoft® Word 2016', 'creationdate': '2024-03-04T15:30:01+09:00', 'moddate': '2024-03-04T15:30:01+09:00', 'source': './Data/2024_KB_부동산_보고서_최종.pdf', 'page_label': '15'}
----------------------------------------
문서 2:
유사 점수 0.8328907591539737:
내용: 18 
2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망 
 
그림Ⅰ-30. 수도권 입주물량과 전세가격 변동률 추이  그림Ⅰ-31. 기타지방 입주물량과 전세가격 변동률 추이 
 
 
 
자료: KB국민은행, 부동산114  자료: KB국민은행, 부동산114...
메타데이터: {'page_label': '25', 'moddate': '2024-03-04T15:30:01+09:00', 'creator': 'Microsoft® Word 2016', 'page': 24, 'source': './Data/2024_KB_부동산_보고서_최종.pdf', 'total_pages': 84, 'producer': 'Microsoft® Word 2016', 'creationdate': '2024-03-04T15:30:01+09:00', 'author': '손은경', 't

In [12]:
# <FAISS>
# 라이브러리 불러오기
from langchain_community.vectorstores import FAISS

In [13]:
# 파이스 DB 생성
faiss_db = FAISS.from_documents(documents=splits,
    embedding=embedding_function
)
print('문서의 수:', faiss_db.index.ntotal)

문서의 수: 135


In [ ]:
# 파이스 DB 저장하기
# faiss_directory = '/content/drive/MyDrive/langchain_tutorial/Ch02. RAG/directory/faiss_index'
faiss_directory = './directory/faiss_index'
faiss_db.save_local(faiss_directory)

# 파이스 DB 불러오기
new_db_faiss = FAISS.load_local(faiss_directory,
				OpenAIEmbeddings(),
				allow_dangerous_deserialization=True)

In [16]:
# 검색할 질문 정의
question = "수도권 주택 매매 전망"

# similarity_search 메서드 사용
docs = new_db_faiss.similarity_search(question)

for i, doc in enumerate(docs, 1):
    print(f"문서 {i}:")
    print(f"내용: {doc.page_content[:150]}...")
    print(f"메타데이터: {doc.metadata}")
    print('--' * 20)

문서 1:
내용: 8 
2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망 
 
실 등에 따른 주택 경기 불안을 이유로 매매를 망설이며 시세 대비 저렴한 매물에만 관심을 보였다. 결
국 매도자와 매수자 간 희망가격 차이로 인한 매매 거래 위축 현상은 2023년 거래 침체의 가...
메타데이터: {'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2024-03-04T15:30:01+09:00', 'title': 'Morning Meeting', 'author': '손은경', 'moddate': '2024-03-04T15:30:01+09:00', 'source': './Data/2024_KB_부동산_보고서_최종.pdf', 'total_pages': 84, 'page': 14, 'page_label': '15'}
----------------------------------------
문서 2:
내용: 18 
2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망 
 
그림Ⅰ-30. 수도권 입주물량과 전세가격 변동률 추이  그림Ⅰ-31. 기타지방 입주물량과 전세가격 변동률 추이 
 
 
 
자료: KB국민은행, 부동산114  자료: KB국민은행, 부동산114...
메타데이터: {'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2024-03-04T15:30:01+09:00', 'title': 'Morning Meeting', 'author': '손은경', 'moddate': '2024-03-04T15:30:01+09:00', 'source': './Data/2024_KB_부동산_보고서_최종.pdf', 'total_pages': 84, 'page': 24, 'page_label': '25'}
--------------------------

In [ ]:
# 검색할 질문 정의
question = "수도권 주택 매매 전망"

# similarity_search 메서드 사용
docs = new_db_faiss.similarity_search(question)

for i, doc in enumerate(docs, 1):
    print(f"문서 {i}:")
    print(f"내용: {doc.page_content[:500]}...")
    print(f"메타데이터: {doc.metadata}")
    print('--' * 20)

문서 1:
내용: 8 
2024 KB 부동산 보고서: 2024년 주택시장 진단과 전망 
 
실 등에 따른 주택 경기 불안을 이유로 매매를 망설이며 시세 대비 저렴한 매물에만 관심을 보였다. 결
국 매도자와 매수자 간 희망가격 차이로 인한 매매 거래 위축 현상은 2023년 거래 침체의 가장 큰 원
인으로 볼 수 있다. 
또한 과거 전국적인 주택 거래 호황기와는 달리 서울 등 일부 지역을 중심으로 매수세가 형성되고, 
전세 사기를 포함한 보증금 미반환 리스크에 따라 비아파트에 대한 선호도가 낮아지면서 아파트로 수
요가 집중되었다. 거시경제 불확실성이 확대되면서 투자 수요 보다 실수요를 중심으로 매매 거래가 이
루어지는 등 다양한 원인이 복합적으로 작용하여 거래 침체를 유발하였다. 
이에 따라 꾸준한 주택 재고 증가에도 불구하고 주택 매매 거래회전율(전체 주택 재고 대비 주택 매
매거래 비율)은 최근 2년간 평균 3%를 하회하면서 지난 10년 전국 평균(6%) 대비 절반 수준에 머물
렀다. 전체 주택시장에...
메타데이터: {'source': '/content/drive/MyDrive/Colab Notebooks/langchain_book/2024_KB_부동산_보고서_최종.pdf', 'page': 14, 'page_label': '15'}
----------------------------------------
문서 2:
내용: 그림Ⅰ-14. 매수우위지수 및 매매가격전망지수 추이  그림Ⅰ-15. 전국 및 서울 주택 매매 거래회전율 추이 
 
 
 
주: 갭=매매가격전망지수-매수우위지수 
자료: KB국민은행  
자료: 부동산114 
 
 
■ 하반기 이후 거래 침체 현상이 다소 완화될 수 있으나 전반적인 회복세는 제한적일 것 
주택 매매 거래 회복을 위해서는 매도자와 매수자 간 희망가격 격차 축소와 매수세 회복이 중요한데, 
여전히 불확실한 주택가격 향방으로 주택시장에 대한 매도자와 매수자의 기대감이 달라 이러한 요인들
이 단기간에 해소되기는 쉽지 않을 것으로 보인다